In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab_Notebooks/BML')

Mounted at /content/drive


In [2]:
!pip install git+https://github.com/betteridiot/seqlogo.git#egg=seqlogo

  Cloning https://github.com/betteridiot/seqlogo.git to /tmp/pip-install-dmw26c0_/seqlogo_12b5c849206c41fcb4c598d69fcb1a4e
  Running command git clone --filter=blob:none --quiet https://github.com/betteridiot/seqlogo.git /tmp/pip-install-dmw26c0_/seqlogo_12b5c849206c41fcb4c598d69fcb1a4e
  Resolved https://github.com/betteridiot/seqlogo.git to commit f265220fbad4962e13a061311b86cee5170c937b
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.7/571.7 kB 10.3 MB/s eta 0:00:00
  Created wheel for seqlogo: filename=seqlogo-5.29.8-py2.py3-none-any.whl size=19401 sha256=e474c42af82f9bdc4f363f18cdf0789a2d26c87394d9c4c92ba8ba3c92e6f852
  Stored in directory: /tmp/pip-ephem-wheel-cache-qzo1u0r7/wheels/af/d5/b2/51372fee875a49d96e034e1457bde673ab2caf297e4870d5a4
Successfully built seqlogo


In [5]:
import numpy as np
import pandas as pd
import seqlogo

CHATGPT 範例檔

In [ ]:
# Setting seed for demonstration purposes
np.random.seed(42)

# Making some fake Position Frequency Data (PFM)
pfm = pd.DataFrame(np.random.randint(0, 36, size=(8, 4)))

# Convert to Position Weight Matrix (PWM)
pwm = seqlogo.pfm2pwm(pfm)
pwm

,A,C,G,T
0,0.698830,-0.301170,-1.301170,0.213404
1,0.263034,0.552541,-0.584962,-0.584962
2,0.148523,0.754244,0.148523,-3.375039
3,0.182864,-4.209453,0.314109,0.648528
4,-4.000000,0.321928,1.000000,-0.540568
5,-0.222392,-0.029747,0.085730,0.140178
6,0.697437,0.597902,-2.209453,-0.624491
7,0.736966,-0.584962,0.502500,-2.000000


In [ ]:
pfm

,0,1,2,3
0,28,14,7,20
1,18,22,10,10
2,23,35,23,2
3,21,1,23,29
4,1,20,32,11
5,21,24,26,27
6,15,14,2,6
7,20,8,17,3


# Negative PWM

2023/11/9 PM 1:35 開始創建 Negative 的 PWM

In [19]:
def parse_fasta(filepath):
  """
  This function is used to deal with fasta file without external library
  """

  with open(filepath, 'r') as file:
    # Initialize variables
    sequence = []
    header = None
    sequence_parts = []

    # Read the FASTA file line by line
    for line in file:
      line = line.strip()
      if line.startswith('>'): # Header line
        if header: # If there is a header, we've reached a new sequence; save the previous one
          sequence.append({'header':header, 'sequence':''.join(sequence_parts)})
        header = line[1:] # Get the header, excluding the '>'
        sequence_parts = [] # Reset the sequence list
      else:
        sequence_parts.append(line) # Sequence line; append to the list

    # After the last header, there will be a sequence to add
    if header and sequence_parts:
      sequence.append({'header':header, 'sequence':''.join(sequence_parts)})
    return sequence

# Parse the fasta file
fasta_sequence = parse_fasta('/content/drive/MyDrive/Colab_Notebooks/BML/Ass2/CD_HIT_Neg.fasta')

# Show the first few entries
fasta_sequence[:5]


[{'header': 'Q2RBM4 Oryza sativa subsp. japonica_17',
  'sequence': 'TRKREELAGEVHDLHKKTRADDEPADDNHTM'},
 {'header': 'Q2RBM4 Oryza sativa subsp. japonica_58',
  'sequence': 'DLHSRQLAVYGRETMKRLFASNVLVSGLNGL'},
 {'header': 'Q2RBM4 Oryza sativa subsp. japonica_79',
  'sequence': 'NVLVSGLNGLGAEIAKNLVLAGVKSVNLHDD'},
 {'header': 'Q2RBM4 Oryza sativa subsp. japonica_161',
  'sequence': 'LSNFQAVVFTDISLEKAVEFDSYCHNHQPPI'},
 {'header': 'Q2RBM4 Oryza sativa subsp. japonica_180',
  'sequence': 'FDSYCHNHQPPIAFIKSEIRGLFGSVFCDFG'}]

In [20]:
df = pd.DataFrame(fasta_sequence)

sequence = df.iloc[:,1]
sequence

0        TRKREELAGEVHDLHKKTRADDEPADDNHTM
1        DLHSRQLAVYGRETMKRLFASNVLVSGLNGL
2        NVLVSGLNGLGAEIAKNLVLAGVKSVNLHDD
3        LSNFQAVVFTDISLEKAVEFDSYCHNHQPPI
4        FDSYCHNHQPPIAFIKSEIRGLFGSVFCDFG
                      ...               
37620    IPRGGPHRACIVANDKLFVIGGQEGDFMAKP
37621    GQEGDFMAKPGSPIFKCVRRHEVVYGDVYML
37622    HEVVYGDVYMLDDGNKWKQLSPLPKPNSHIE
37623    VIVNNSVIIVGGTTEKHPITKKMILVGEVFR
37624    DTLTWSVIGRMPFRIKTALAGYWDGWLYFTS
Name: sequence, Length: 37625, dtype: object

## 創建 Matrix for Mapping

In [21]:
import numpy as np

# Create a 21x31 dimensional matrix
matrix = np.zeros((21, 31))

matrix.shape, matrix

((21, 31),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0

In [22]:
# mapping 21 amino acid(including X) to the row of matrix
aa_list = 'ACDEFGHIKLMNPQRSTVWY-'

aa2row = {aa: index for index, aa in enumerate(aa_list)} # create a dict for aa

# Define the function to update the matrix
def update_matrix(matrix, sequence, aa2row):
  for position, amino_acid in enumerate(sequence):
    if amino_acid in aa2row:
      row_index = aa2row[amino_acid]
      matrix[row_index, position] += 1 # Update the matrix


In [23]:
for seq in sequence: # 這樣就可以遍歷一個df
    update_matrix(matrix, seq, aa2row)


## PFM

In [24]:
matrix # 數字後面有點表示為float(浮點數)

array([[ 3012.,  2928.,  2971.,  3037.,  3018.,  3036.,  3053.,  2907.,
         2903.,  3064.,  3067.,  2891.,  3081.,  3099.,  2992.,     0.,
         2801.,  2879.,  2773.,  2790.,  2939.,  2918.,  2804.,  2822.,
         2876.,  2905.,  2869.,  2832.,  2914.,  2776.,  2981.],
       [  613.,   536.,   546.,   602.,   556.,   571.,   574.,   602.,
          573.,   622.,   572.,   564.,   575.,   598.,   618.,     0.,
          546.,   592.,   515.,   502.,   556.,   563.,   531.,   519.,
          545.,   580.,   569.,   514.,   554.,   568.,   547.],
       [ 2284.,  2278.,  2182.,  2263.,  2378.,  2273.,  2267.,  2206.,
         2385.,  2238.,  2294.,  2522.,  2553.,  2261.,  2111.,     0.,
         2213.,  1859.,  2280.,  2312.,  2245.,  2098.,  2286.,  2320.,
         2166.,  2134.,  2237.,  2204.,  2134.,  2147.,  2187.],
       [ 2810.,  2927.,  2937.,  2933.,  2917.,  2905.,  2823.,  2937.,
         3063.,  2934.,  2872.,  3417.,  3635.,  2942.,  3417.,     0.,
         3015

In [25]:
df_matrix = pd.DataFrame(matrix, index=list(aa_list))
df_matrix

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
A,3012.0,2928.0,2971.0,3037.0,3018.0,3036.0,3053.0,2907.0,2903.0,3064.0,...,2918.0,2804.0,2822.0,2876.0,2905.0,2869.0,2832.0,2914.0,2776.0,2981.0
C,613.0,536.0,546.0,602.0,556.0,571.0,574.0,602.0,573.0,622.0,...,563.0,531.0,519.0,545.0,580.0,569.0,514.0,554.0,568.0,547.0
D,2284.0,2278.0,2182.0,2263.0,2378.0,2273.0,2267.0,2206.0,2385.0,2238.0,...,2098.0,2286.0,2320.0,2166.0,2134.0,2237.0,2204.0,2134.0,2147.0,2187.0
E,2810.0,2927.0,2937.0,2933.0,2917.0,2905.0,2823.0,2937.0,3063.0,2934.0,...,2758.0,2978.0,2933.0,2943.0,2871.0,2803.0,2898.0,2952.0,2912.0,2833.0
F,1328.0,1439.0,1390.0,1428.0,1372.0,1423.0,1464.0,1442.0,1520.0,1469.0,...,1424.0,1378.0,1294.0,1370.0,1414.0,1354.0,1308.0,1428.0,1378.0,1464.0
G,2582.0,2624.0,2577.0,2590.0,2669.0,2682.0,2555.0,2631.0,2526.0,2619.0,...,2522.0,2494.0,2423.0,2568.0,2588.0,2560.0,2479.0,2483.0,2572.0,2502.0
H,792.0,776.0,860.0,860.0,859.0,872.0,792.0,858.0,803.0,824.0,...,825.0,792.0,829.0,819.0,820.0,782.0,784.0,809.0,816.0,800.0
I,1826.0,1892.0,1889.0,1780.0,1825.0,1798.0,1873.0,1893.0,1873.0,1889.0,...,1907.0,1821.0,1808.0,1833.0,1906.0,1801.0,1851.0,1891.0,1858.0,1860.0
K,1839.0,1879.0,1793.0,1743.0,1724.0,1631.0,1459.0,1481.0,1366.0,1206.0,...,2528.0,2713.0,2708.0,2556.0,2681.0,2679.0,2648.0,2542.0,2647.0,2574.0
L,3509.0,3413.0,3523.0,3336.0,3389.0,3432.0,3565.0,3466.0,3461.0,3584.0,...,3466.0,3301.0,3363.0,3475.0,3367.0,3473.0,3351.0,3379.0,3385.0,3351.0


In [26]:
# Resetting the column names from -15 to +15
new_columns = list(range(-15, 16))
df_matrix.columns = new_columns

df_matrix  # Display the DataFrame to verify the new column names


,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,...,6,7,8,9,10,11,12,13,14,15
A,3012.0,2928.0,2971.0,3037.0,3018.0,3036.0,3053.0,2907.0,2903.0,3064.0,...,2918.0,2804.0,2822.0,2876.0,2905.0,2869.0,2832.0,2914.0,2776.0,2981.0
C,613.0,536.0,546.0,602.0,556.0,571.0,574.0,602.0,573.0,622.0,...,563.0,531.0,519.0,545.0,580.0,569.0,514.0,554.0,568.0,547.0
D,2284.0,2278.0,2182.0,2263.0,2378.0,2273.0,2267.0,2206.0,2385.0,2238.0,...,2098.0,2286.0,2320.0,2166.0,2134.0,2237.0,2204.0,2134.0,2147.0,2187.0
E,2810.0,2927.0,2937.0,2933.0,2917.0,2905.0,2823.0,2937.0,3063.0,2934.0,...,2758.0,2978.0,2933.0,2943.0,2871.0,2803.0,2898.0,2952.0,2912.0,2833.0
F,1328.0,1439.0,1390.0,1428.0,1372.0,1423.0,1464.0,1442.0,1520.0,1469.0,...,1424.0,1378.0,1294.0,1370.0,1414.0,1354.0,1308.0,1428.0,1378.0,1464.0
G,2582.0,2624.0,2577.0,2590.0,2669.0,2682.0,2555.0,2631.0,2526.0,2619.0,...,2522.0,2494.0,2423.0,2568.0,2588.0,2560.0,2479.0,2483.0,2572.0,2502.0
H,792.0,776.0,860.0,860.0,859.0,872.0,792.0,858.0,803.0,824.0,...,825.0,792.0,829.0,819.0,820.0,782.0,784.0,809.0,816.0,800.0
I,1826.0,1892.0,1889.0,1780.0,1825.0,1798.0,1873.0,1893.0,1873.0,1889.0,...,1907.0,1821.0,1808.0,1833.0,1906.0,1801.0,1851.0,1891.0,1858.0,1860.0
K,1839.0,1879.0,1793.0,1743.0,1724.0,1631.0,1459.0,1481.0,1366.0,1206.0,...,2528.0,2713.0,2708.0,2556.0,2681.0,2679.0,2648.0,2542.0,2647.0,2574.0
L,3509.0,3413.0,3523.0,3336.0,3389.0,3432.0,3565.0,3466.0,3461.0,3584.0,...,3466.0,3301.0,3363.0,3475.0,3367.0,3473.0,3351.0,3379.0,3385.0,3351.0


## PFM2PWM

In [27]:
df_matrix_norm = df_matrix /37625 # df可以直接做運算 :O
df_matrix_norm

,-15,-14,-13,-12,-11,-10,-9,-8,-7,-6,...,6,7,8,9,10,11,12,13,14,15
A,0.080053,0.077821,0.078963,0.080718,0.080213,0.080691,0.081143,0.077262,0.077156,0.081435,...,0.077555,0.074525,0.075003,0.076439,0.077209,0.076252,0.075269,0.077449,0.073781,0.079229
C,0.016292,0.014246,0.014512,0.016000,0.014777,0.015176,0.015256,0.016000,0.015229,0.016532,...,0.014963,0.014113,0.013794,0.014485,0.015415,0.015123,0.013661,0.014724,0.015096,0.014538
D,0.060704,0.060545,0.057993,0.060146,0.063203,0.060412,0.060252,0.058631,0.063389,0.059482,...,0.055761,0.060757,0.061661,0.057568,0.056718,0.059455,0.058578,0.056718,0.057063,0.058126
E,0.074684,0.077794,0.078060,0.077953,0.077528,0.077209,0.075030,0.078060,0.081409,0.077980,...,0.073302,0.079150,0.077953,0.078219,0.076306,0.074498,0.077023,0.078458,0.077395,0.075296
F,0.035296,0.038246,0.036944,0.037953,0.036465,0.037821,0.038910,0.038326,0.040399,0.039043,...,0.037847,0.036625,0.034392,0.036412,0.037581,0.035987,0.034764,0.037953,0.036625,0.038910
G,0.068625,0.069741,0.068492,0.068837,0.070937,0.071282,0.067907,0.069927,0.067136,0.069608,...,0.067030,0.066286,0.064399,0.068252,0.068784,0.068040,0.065887,0.065993,0.068359,0.066498
H,0.021050,0.020625,0.022857,0.022857,0.022831,0.023176,0.021050,0.022804,0.021342,0.021900,...,0.021927,0.021050,0.022033,0.021767,0.021794,0.020784,0.020837,0.021502,0.021688,0.021262
I,0.048532,0.050286,0.050206,0.047309,0.048505,0.047787,0.049781,0.050312,0.049781,0.050206,...,0.050684,0.048399,0.048053,0.048718,0.050658,0.047867,0.049196,0.050259,0.049382,0.049435
K,0.048877,0.049940,0.047654,0.046326,0.045821,0.043349,0.038777,0.039362,0.036306,0.032053,...,0.067189,0.072106,0.071973,0.067934,0.071256,0.071203,0.070379,0.067561,0.070352,0.068412
L,0.093262,0.090711,0.093635,0.088664,0.090073,0.091216,0.094751,0.092120,0.091987,0.095256,...,0.092120,0.087734,0.089382,0.092359,0.089488,0.092306,0.089063,0.089807,0.089967,0.089063


## Download the DataFrame

In [28]:
df_matrix_norm.to_csv('/content/drive/MyDrive/Colab_Notebooks/BML/Ass2/df_matrix_norm_neg.csv')

In [29]:
df_matrix_norm.shape

(21, 31)